## PyTorch实现MNIST手写数字识别

In [1]:
import torch
import torchvision
from torch.utils.data import DataLoader

### 1.获取数据


In [2]:
train_data = torchvision.datasets.MNIST(
    root = "../../../datas/",
    train = True,
    download = True,
    transform = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean = [0.5],std = [0.5])
    ])

) 

In [3]:
test_data = torchvision.datasets.MNIST(
    root = "../../../datas/",
    train = False,
    download = True,
    transform = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean = [0.5],std = [0.5])
    ])
) 

In [4]:
train_data.__len__()

60000

In [5]:
test_data.__len__()

10000

In [6]:
img , target = train_data[0]
img

tensor([[[-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1

### 2.加载数据

In [7]:
train_load = DataLoader(train_data,batch_size=64,shuffle=True,num_workers=4,pin_memory=True)
test_load = DataLoader(test_data,batch_size=64,num_workers=4,pin_memory=True)

### 3. 构建神经网络

In [8]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.model1 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=1,out_channels=64,kernel_size=3,stride=1,padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(64,128,kernel_size=3,stride=1,padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(stride=2,kernel_size=2),
        )
        self.model2 = torch.nn.Sequential(
            torch.nn.Linear(14*14*128,1024),
            torch.nn.ReLU(),
            torch.nn.Dropout(p=0.3),
            torch.nn.Linear(1024, 10)
        )
    def forward(self,x):
        x = self.model1(x)
        x = x.view(-1, 14*14*128)
        x = self.model2(x)
        return x

### 4.损失函数

In [9]:
loss_fn = torch.nn.CrossEntropyLoss()

### 5. 优化器

In [10]:
model = Model()
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=10,gamma=0.1)

### 6.训练

In [11]:
epoch = 10
test_data_size = len(test_data)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
loss_fn.to(device)

CrossEntropyLoss()

In [12]:
for i in range(epoch):
    print("-------------第{}轮训练开始------------".format(i+1))
    # 训练步骤开始
    model.train()
    for data in train_load:
        imgs,targets = data
        imgs = imgs.to(device)
        targets = targets.to(device)
        outputs = model(imgs)
        loss = loss_fn(outputs,targets)
        
        # 优化器优化模型
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print("训练轮数：{},loss:{}".format(i,loss.item()))
        
    # 测试步骤
    model.eval()
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data in test_load:
            imgs,targets = data
            imgs = imgs.to(device)
            targets = targets.to(device)
            outputs = model(imgs)
            loss = loss_fn(outputs,targets)
            total_test_loss += loss.item()
            accuracy = (outputs.argmax(1) == targets).sum()
            total_accuracy+=accuracy
    print("整体测试集上的loss:{}".format(total_test_loss))
    print("整体测试集上的正确率:{}".format(total_accuracy/test_data_size))

    # 保存模型
    if i == epoch-1:
        torch.save(model.state_dict(),"./models/minist_model.pth".format(i))
        print("模型已保存")


-------------第1轮训练开始------------
训练轮数：0,loss:0.3863140642642975
整体测试集上的loss:23.330865898169577
整体测试集上的正确率:0.9554999470710754
-------------第2轮训练开始------------
训练轮数：1,loss:0.10280429571866989
整体测试集上的loss:20.455924058303935
整体测试集上的正确率:0.9612999558448792
-------------第3轮训练开始------------
训练轮数：2,loss:0.19434784352779388
整体测试集上的loss:23.062405189150013
整体测试集上的正确率:0.9527999758720398
-------------第4轮训练开始------------
训练轮数：3,loss:0.0782708153128624
整体测试集上的loss:19.928659476485336
整体测试集上的正确率:0.9652999639511108
-------------第5轮训练开始------------
训练轮数：4,loss:0.06367688626050949
整体测试集上的loss:18.819381162022182
整体测试集上的正确率:0.9673999547958374
-------------第6轮训练开始------------
训练轮数：5,loss:0.006555695086717606
整体测试集上的loss:20.897021545286407
整体测试集上的正确率:0.9656999707221985
-------------第7轮训练开始------------
训练轮数：6,loss:0.010882716625928879
整体测试集上的loss:22.689026294136056
整体测试集上的正确率:0.9651999473571777
-------------第8轮训练开始------------
训练轮数：7,loss:0.01195739395916462
整体测试集上的loss:22.503557900199667
整体测试集上的正确率:0.967299997

### 7.模型预测

In [13]:
# 模型加载
model = Model()
model_status = torch.load("./models/minist_model.pth")
model.load_state_dict(model_status)

<All keys matched successfully>

In [14]:
img,target = test_data[0]

In [15]:
img

tensor([[[-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
          -1.0000, -1.0000, -1.0000, -1.0000, -1

In [16]:
target

7

In [17]:
output = model(img)
output

tensor([[-16.7447,  -9.4313,  -6.2096,  -3.1663, -14.6242,  -6.6308, -24.7730,
          15.9027,  -9.5621,  -5.7777]], grad_fn=<AddmmBackward0>)

In [18]:
output.argmax(1)

tensor([7])